# *Controllers*

In [2]:
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pmdarima as pm
from statsforecast import StatsForecast

from utilsforecast.losses import mse
from statsforecast.models import (
    HoltWinters,
    AutoCES,
    AutoARIMA,
    AutoTheta,
    SeasonalNaive
)

CURRENT_DIR = Path().resolve()

BASE_DIR = CURRENT_DIR.parents[1]

DATA_DIR = BASE_DIR / "data" / "raw"

FIGURES_DIR = BASE_DIR / "notebooks" / "figures"
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

RESULTS_DIR = BASE_DIR / "notebooks" / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

models = [
    HoltWinters(),
    AutoCES(season_length = 12),
    AutoARIMA(season_length = 12),
    AutoTheta(season_length = 12),
    SeasonalNaive(season_length = 12)
]

file_paths = sorted(DATA_DIR.glob("Presupuesto*.xlsx"))

categories = ['Ingresos (+)', 'Egresos (-)']

income_details = [
    '019 - Servicios Mecánicos',
    '020 - Servicios Eléctricos',
    '021 - Servicios Instrumentación',
    '022 - Transferencias Estado',
    '024 - Ingresos Financieros',
    '025 - Intereses',
    '030 - Saldo Inicial de Caja',
]

expense_details = [
    '101 - Sueldos Personal',
    '102 - Bonos Personal',
    '103 - Seguridad Social',
    '107 - Servicios Externos',
    '110 - Seguros',
    '117 - Servicios TI',
    '129 - Inversión Financiera',
    '130 - Saldo Final de Caja'
]

excluded_projects = [
    '120 - Marketing',
    '121 - Logística',
    '123 - Viáticos',
    '126 - Compras Activos'
]

final_columns = [
    'Servicios Mecánicos',
    'Servicios Eléctricos',
    'Servicios Instrumentación',
    'Transferencias Estado',
    'Ingresos Financieros',
    'Intereses',
    'Saldo Inicial de Caja',
    'Otros Ingresos',
    'Sueldos Personal',
    'Bonos Personal',
    'Seguridad Social',
    'Servicios Externos',
    'Seguros',
    'Servicios TI',
    'Otros Proyectos',
    'Inversión Financiera',
    'Saldo Final de Caja',
    'Otros Egresos'
]

income_detail_names = [x[6:] for x in income_details]

expense_detail_names = [x[6:] for x in expense_details]

# *build_general_dataset*

In [3]:
def build_general_dataset(file_paths, category_name, master_df):
    for i in range(len(file_paths)):

        data = pd.read_excel(file_paths[i])
        data = data.set_index('Unnamed: 0')
        data.index = data.index.str.strip()
        data = data.loc[: , 'Enero' : 'Diciembre']
        data = data.drop('Detalle').T

        lower_bound = i * 12
        upper_bound = i * 12 + 11

        if category_name == 'Ingresos (+)':
            master_df.loc[lower_bound : upper_bound, 'Ingresos'] = data[category_name].tolist()

            for j in range(len(income_details)):
                master_df.loc[lower_bound : upper_bound, income_detail_names[j]] = data[income_details[j]].tolist()

            other_income = data.loc[: , '001 - Ventas Servicios Industriales' : '030 - Saldo Inicial de Caja']
            other_income = other_income.drop(income_details, axis = 1)
            other_income = other_income.drop('029 - Ventas de Activos', axis = 1)

            master_df.loc[lower_bound : upper_bound, 'Otros Ingresos'] = other_income.sum(axis = 1).tolist()

        if category_name == 'Egresos (-)':
            master_df.loc[lower_bound : upper_bound, 'Egresos'] = data[category_name].tolist()

            for j in range(len(expense_details)):
                master_df.loc[lower_bound : upper_bound, expense_detail_names[j]] = data[expense_details[j]].tolist()

            other_projects = data.loc[: , '119 - Consultorías' : '128 - Provisiones']
            other_projects = other_projects.drop(excluded_projects, axis = 1)
            other_expenses = data.loc[: , '101 - Sueldos Personal' : '130 - Saldo Final de Caja']
            other_expenses = other_expenses.drop(expense_details, axis = 1).drop(other_projects.columns.tolist(), axis = 1)

            master_df.loc[lower_bound : upper_bound, 'Otros Proyectos'] = other_projects.sum(axis = 1).tolist()
            master_df.loc[lower_bound : upper_bound, 'Otros Egresos'] = other_expenses.sum(axis = 1).tolist()

# *build_detailed_dataset*

In [4]:
def build_detailed_dataset(file_paths, general_df):
    data = general_df.drop(['Ingresos', 'Egresos'], axis = 1)

    series_names = data.drop('Fecha', axis = 1).columns.to_list()
    total_months = len(file_paths) * 12
    total_records = len(series_names) * total_months

    detailed_df = pd.DataFrame({'Numero': np.arange(0, total_records)})
    detailed_df = detailed_df.set_index('Numero')

    for i in range(len(series_names)):
        lim_inf = i * total_months
        lim_sup = i * total_months + total_months - 1

        detailed_df.loc[lim_inf : lim_sup, 'ds'] = data['Fecha'].to_list()
        detailed_df.loc[lim_inf : lim_sup, 'unique_id'] = series_names[i]
        detailed_df.loc[lim_inf : lim_sup, 'y'] = data[series_names[i]].to_list()

    return detailed_df

# *General Forecasting Model*
- generate_general_forecast
- build_general_model

In [5]:
def generate_general_forecast(general_model, test, figures_dir):
    forecast, forecast_ci = general_model.predict(n_periods = 72, return_conf_int = True)
    forecast_ci = pd.DataFrame(forecast_ci)
    forecast_ci = forecast_ci.set_index(forecast.index)

    plt.figure(figsize = (20, 8))
    ax = test.plot(label = 'Observed Data')
    forecast.plot(label = 'General AutoARIMA Model')

    ax.fill_between(
        forecast_ci.index,
        forecast_ci.iloc[:, 0],
        forecast_ci.iloc[:, 1],
        color = 'red',
        alpha = 0.2,
        label = 'Confidence Interval [95%]'
    )

    plt.title('General AutoARIMA Model vs Observed Data (2025)')
    plt.grid(True)
    plt.legend()

    save_path = figures_dir / "general_forecast.png"
    plt.savefig(save_path, dpi = 300, bbox_inches = "tight")
    plt.close()

    forecast.index.name = 'Fecha'
    forecast = forecast.rename('General Model')

    return forecast

In [6]:
def build_general_model(general_df, file_paths, figures_dir):
    split_index = len(file_paths) * 12 - 12

    data = general_df
    data = data.set_index('Fecha')

    train = data['Ingresos'][:split_index]
    test = data['Ingresos'][split_index:]

    model = pm.auto_arima(
        train,
        m = 12,
        seasonal = True,
        trace = False,
        stepwise = True,
        max_p = 3, max_q = 3
    )

    print(model.summary())

    forecast = generate_general_forecast(model, test, figures_dir)

    return model, forecast

# *generate_detailed_forecast*

- evaluate_cv
- get_best_model_forecast
- generate_detailed_forecast

In [7]:
def evaluate_cv(df, metric):
    models = df.columns.drop(['unique_id', 'ds', 'y', 'cutoff']).tolist()
    evals = metric(df, models = models)
    evals['best_model'] = evals[models].idxmin(axis=1)
    return evals

In [8]:
def get_best_model_forecast(forecasts_df, evaluation_df):
    with_best = forecasts_df.merge(evaluation_df[['unique_id', 'best_model']])
    res = with_best[['unique_id', 'ds']].copy()
    for suffix in ('', '-lo-95', '-hi-95'):
        res[f'best_model{suffix}'] = with_best.apply(lambda row: row[row['best_model'] + suffix], axis = 1)
    return res

In [9]:
def generate_detailed_forecast(detailed_df, models, figures_dir):
    train = detailed_df[detailed_df.ds < '2025-01-01']

    sf = StatsForecast(
        models = models,
        freq = 'ME',
        fallback_model = SeasonalNaive(season_length = 12),
        n_jobs = -1
    )

    forecasts_df = sf.forecast(df = train, h = 72, level = [95])

    cv_df = sf.cross_validation(
        df = train,
        h = 12,
        step_size = 12,
        n_windows = 1
    )

    evaluation_df = evaluate_cv(cv_df, mse)
    detailed_forecast = get_best_model_forecast(forecasts_df, evaluation_df)

    fig = sf.plot(train, detailed_forecast, level = [95])

    save_path = figures_dir / "detailed_forecasts.png"
    fig.savefig(save_path, dpi = 300)
    plt.close(fig)

    return detailed_forecast

# *Final Financial Statement Builder*

- compute_annual_asset_sales
- build_final_financial_statement

In [10]:
def compute_annual_asset_sales(detailed_forecast, general_df, final_dates):
    financial_investment_forecast = detailed_forecast[detailed_forecast.unique_id == 'Inversión Financiera']
    years = pd.unique(financial_investment_forecast.ds.dt.year)

    total_months = (len(years) - 1) * 12
    asset_sales = [0] * total_months

    investment_2024 = general_df[['Fecha', 'Inversión Financiera']]
    investment_2024 = investment_2024[investment_2024['Fecha'].dt.year == 2024]
    investment_2024 = investment_2024['Inversión Financiera'].sum()

    asset_sales[0] = investment_2024

    for i in range(len(years) - 1):
        lower_bound = i * 12
        upper_bound = i * 12 + 12

        annual_investment = financial_investment_forecast.iloc[lower_bound : upper_bound].best_model.sum()

        if upper_bound < total_months:
            asset_sales[upper_bound] = annual_investment

    asset_sales_df = pd.DataFrame({'Fecha': final_dates})
    asset_sales_df = asset_sales_df.set_index('Fecha')
    asset_sales_df['Ventas de Activos'] = asset_sales

    return asset_sales_df

In [11]:
def build_final_financial_statement(general_forecast, detailed_forecast, final_dates, asset_sales_forecast, final_columns):
    df = detailed_forecast[['unique_id', 'ds', 'best_model']]

    temp_df = pd.DataFrame({'Fecha': final_dates})
    temp_df = temp_df.set_index('Fecha')

    final_df = pd.DataFrame({'Fecha': final_dates})
    final_df = final_df.set_index('Fecha')

    for i in range(len(final_columns)):
        series_data = df[df.unique_id == final_columns[i]]

        temp_df[final_columns[i]] = series_data['best_model'].to_list()

    final_df['Ingresos'] = (temp_df[final_columns[:8]].sum(axis = 1) + asset_sales_forecast['Ventas de Activos']).to_list()
    final_df['Ventas de Activos'] = asset_sales_forecast['Ventas de Activos'].to_list()
    final_df = pd.concat([final_df, temp_df[final_columns[:8]]], axis = 1)

    final_df['Egresos'] = temp_df[final_columns[8:]].sum(axis = 1).to_list()
    final_df = pd.concat([final_df, temp_df[final_columns[8:]]], axis = 1)

    final_df['General Model'] = general_forecast.to_list()

    return final_df.T

# *Annual Cash Flow Model*

- Flujo_Caja_Anual
- Saldo_Inicial_Caja

In [12]:
def compute_initial_cash_balance(final_statement, annual_dates):
    data = final_statement.loc[['Ventas de Activos', 'Saldo Inicial de Caja']]

    initial_cash = []

    for i in range(len(annual_dates)):
        initial_cash.append(data[pd.Timestamp(annual_dates[i])].sum())

    return initial_cash

In [13]:
def build_annual_cash_flow(final_df):
    df = final_df.copy()

    annual_dates = np.arange('2025-01-01', '2031-01-01', dtype = 'datetime64[Y]')

    data = df.drop(['Ventas de Activos', 'Saldo Inicial de Caja', 'Inversión Financiera', 'Saldo Final de Caja'])

    date_labels = pd.to_datetime(data.columns, errors = 'coerce')
    years = date_labels.year.unique()

    annual_income = []
    annual_expenses = []

    for i in range(len(years)):
        lower_bound = i * 12
        upper_bound = i * 12 + 12

        yearly_segment = data.iloc[: , lower_bound : upper_bound]

        annual_income.append(yearly_segment.loc['Servicios Mecánicos' : 'Otros Ingresos'].sum(axis = 1).sum())
        annual_expenses.append(yearly_segment.loc['Sueldos Personal' : 'Otros Egresos'].sum(axis = 1).sum())

    initial_cash = compute_initial_cash_balance(df, annual_dates)

    annual_cash_flow = pd.DataFrame({
        'Saldo Inicial de Caja': initial_cash,
        'Ingresos': annual_income,
        'Egresos': annual_expenses}
    )

    annual_cash_flow['Saldo Final de Caja'] = (
            annual_cash_flow['Saldo Inicial de Caja']
            + annual_cash_flow['Ingresos']
            - annual_cash_flow['Egresos']
    )

    annual_cash_flow = annual_cash_flow.set_index(annual_dates)

    return annual_cash_flow.T

# *Main*

In [14]:
# ======================
# Main Execution Pipeline
# ======================

if __name__ == "__main__":

    # -----------------------------
    # Date Ranges
    # -----------------------------
    historical_dates = np.arange(
        '2013-01',
        '2026-01',
        dtype='datetime64'
    )

    forecast_dates = np.arange(
        '2025-01',
        '2031-01',
        dtype='datetime64'
    )

    # -----------------------------
    # Build General Dataset
    # -----------------------------
    general_df = pd.DataFrame({'Fecha': historical_dates})

    for i in range(len(categories)):
        build_general_dataset(
            file_paths,
            categories[i],
            general_df
        )

    # -----------------------------
    # Build Detailed Dataset
    # -----------------------------
    detailed_df = build_detailed_dataset(
        file_paths,
        general_df
    )

    # -----------------------------
    # General Model Forecast
    # -----------------------------
    general_model, general_forecast = build_general_model(
        general_df,
        file_paths,
        FIGURES_DIR
    )

    # -----------------------------
    # Detailed Multi-Series Forecast
    # -----------------------------
    detailed_forecast = generate_detailed_forecast(
        detailed_df,
        models,
        FIGURES_DIR
    )

    # -----------------------------
    # Annual Asset Sales
    # -----------------------------
    asset_sales_forecast = compute_annual_asset_sales(
        detailed_forecast,
        general_df,
        forecast_dates
    )

    # -----------------------------
    # Final Financial Statement
    # -----------------------------
    final_statement = build_final_financial_statement(
        general_forecast,
        detailed_forecast,
        forecast_dates,
        asset_sales_forecast,
        final_columns
    )

    # -----------------------------
    # Annual Cash Flow Projection
    # -----------------------------
    annual_cash_flow = build_annual_cash_flow(
        final_statement
    )

    # -----------------------------
    # Export Results
    # -----------------------------
    final_statement.to_excel(
        RESULTS_DIR / "Financial_Forecast_2025_2030.xlsx"
    )

    annual_cash_flow.to_excel(
        RESULTS_DIR / "Annual_Cash_Flow_2025_2030.xlsx"
    )

                                     SARIMAX Results                                      
Dep. Variable:                                  y   No. Observations:                  144
Model:             SARIMAX(1, 0, 0)x(2, 1, 0, 12)   Log Likelihood               -2409.884
Date:                            Wed, 25 Feb 2026   AIC                           4827.768
Time:                                    22:03:50   BIC                           4839.299
Sample:                                01-01-2013   HQIC                          4832.454
                                     - 12-01-2024                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.5069      0.054      9.381      0.000       0.401       0.613
ar.S.L12      -0.2980      0.031   